### Import reviews

In [1]:
%load_ext autoreload
%autoreload 2

import progressbar

In [2]:
import pickle

DATA_FILE1 = 'data/data.pkl'
DATA_FILE2 = 'data/labels.pkl'
DATA_FILE3 = 'data/pos_adjectives_list.pkl'
DATA_FILE4 = 'data/neu_adjectives_list.pkl'
DATA_FILE5 = 'data/neg_adjectives_list.pkl'

with open(DATA_FILE1, 'rb') as f:
    reviews = pickle.load(f)
    
with open(DATA_FILE2, 'rb') as f:
    labels = pickle.load(f)
'''
with open(DATA_FILE3, 'rb') as f:
    pos_adjectives_list = pickle.load(f)
    
with open(DATA_FILE4, 'rb') as f:
    neu_adjectives_list = pickle.load(f)

with open(DATA_FILE5, 'rb') as f:
    neg_adjectives_list = pickle.load(f)
'''    
entity_names = ['X', 'Y', 'Z', 'T', 'P', 'Q']

In [3]:
from babble.babbler import BabbleStream

babbler = BabbleStream(reviews,labels,balanced=True, shuffled=True, seed=456)

Grammar construction complete.


In [ ]:
from babble.utils import display_candidate

candidate = babbler.next()
print(candidate)
print(candidate.text)
print(candidate.entity_positions)

In [4]:
import re

def get_words_before(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity)
        before = index - min(index, quantity)
        return ' '.join(map(str, words[before:index])) 

In [5]:
import re

def get_words_after(quantity,sentence,entity):
    sentence = re.sub(r'[^\w\s]','',sentence)
    words = sentence.split()
    if entity in words:
        index = words.index(entity) +1
        after = index + min(index, quantity)
        return ' '.join(map(str, words[index:after]))

In [ ]:
def check_adjectives_before(sentence,entity):
    words = get_words_before(sentence,entity)
    if words == None:
        return None
    else:
        for word in words:
            if word in pos_adjectives_list:
                return 'positive'
            elif word in neu_adjectives_list:
                return 'neutral'
            elif word in neg_adjectives_list:
                return 'negative'
        return None

In [6]:
def check_adjectives_before_pos(sentence,entity):
    words = get_words_before(2,sentence,entity)
    if words == None or len(words.split(" ")) == 0:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        for token in doc:
            if token.pos_ == 'ADJ':
                return token.text
                break

In [7]:
def check_adjectives_after_verb(sentence,entity):
    words = get_words_after(3,sentence,entity)
    if words == None or len(words.split(" ")) < 3:
        return None
    else:
        spacy_nlp = spacy.load('en_core_web_sm')
        doc = spacy_nlp(words)
        if doc[0].pos_ == 'VERB' and (doc[1].pos_ == 'ADJ' or doc[2].pos_ == 'ADJ'):
            return doc[1].text

In [ ]:
print(get_words_before(2,candidate.text,"chicken"))

In [ ]:
print(check_adjectives_before_pos(candidate.text,"customer"))

In [8]:
import spacy

spacy_stopwords = spacy.lang.en.stop_words.STOP_WORDS

print('spaCy Version: %s' % (spacy.__version__))
spacy_nlp = spacy.load('en_core_web_sm')

spaCy Version: 2.1.8


In [9]:
from babble import Explanation

explanations = []
index = 1

for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
#    print(candidate.text)
    for entity,name in zip(candidate.entities,entity_names):
#        print(entity.entity)
        adjectives_value = check_adjectives_before_pos(candidate.text,entity.entity)
        if adjectives_value != None:
            explanation = Explanation(
            name='LF_' + str(index),
            label=label,
            condition='the word "' + adjectives_value + '" is within two words to the left of ' + name, 
            candidate=candidate
            )
            explanations.append(explanation)
            index = index + 1
#            print(str(explanation).upper())

/ |                                           #    | 6888 Elapsed Time: 2:08:33

KeyboardInterrupt: 

In [ ]:
from babble import Explanation

for candidate,label in progressbar.progressbar(zip(reviews[0],labels[0])):
    for entity,name in zip(candidate.entities,entity_names):
        adjectives_value = check_adjectives_after_verb(candidate.text,entity.entity)
        if adjectives_value != None:
            explanation = Explanation(
            name='LF_'+str(index),
            label=label,
            condition='The word "' + adjectives_value + '" is after ' + name,
            candidate=candidate
            )
            explanations.append(explanation)
            index = index + 1

In [ ]:
print(explanations)

In [ ]:
babbler.view_parse(parses[1])

In [ ]:
print(parses[10].semantics)

In [ ]:
semantics = [parse.semantics for parse in parses ]

In [ ]:
if ('.root', ('.label', ('.int', 1), ('.call', ('.in', ('.extract_text', ('.left', ('.arg', ('.int', 1))))), ('.string', 'average')))) in semantics:
    print("ok")

In [ ]:
print(reviews[0][5].text)

In [ ]:
parses[0].function(reviews[0][0])

In [ ]:
parses, filtered = babbler.apply(explanations)

In [ ]:
babbler.commit()

In [ ]:
from metal.analysis import lf_summary

Ls = [babbler.get_label_matrix(split) for split in [0,1,2]]
lf_names = [lf.__name__ for lf in babbler.get_lfs()]
lf_summary(Ls[1], labels[1], lf_names=lf_names)

In [ ]:
print(Ls)

In [10]:
from babble.utils import ExplanationIO

FILE = "data/my_explanations.tsv"
exp_io = ExplanationIO()
exp_io.write(explanations, FILE)
explanations = exp_io.read(FILE)

Wrote 7739 explanations to data/my_explanations.tsv
Read 7739 explanations from data/my_explanations.tsv
